In [1]:
#import pygraphviz as pgv
from tqdm import tqdm
#from IPython.display import Image

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import tensorflow_gnn as tfgnn
#import tensorflow_datasets as tfds

from tensorflow_gnn import runner
from tensorflow_gnn.models import gat_v2

import networkx as nx
import numpy as np

#import pandas as pd
from pandas import read_parquet as rp

print(f'Using TensorFlow v{tf.__version__} and TensorFlow-GNN v{tfgnn.__version__}')
print(f'GPUs available: {tf.config.list_physical_devices("GPU")}')

import knndatap

Using TensorFlow v2.9.1 and TensorFlow-GNN v0.3.0.dev1
GPUs available: []


ModuleNotFoundError: No module named 'knndatap'

In [ ]:
def makeGT(id,counter):
    print("Generating GraphTensor for this event: ID =",id)
    hit_num = len(simu.loc[id][2]['sensor_pos_x'])+len(simu.loc[id][3]['sensor_pos_x'])
    hits = np.array([np.append(simu.loc[id][2]['t'],simu.loc[id][3]['t']),np.append(simu.loc[id][2]['sensor_pos_x'],simu.loc[id][3]['sensor_pos_x']),np.append(simu.loc[id][2]['sensor_pos_y'],simu.loc[id][3]['sensor_pos_y']),np.append(simu.loc[id][2]['sensor_pos_z'],simu.loc[id][3]['sensor_pos_z']),np.append(simu.loc[id][2]['string_id'],simu.loc[id][3]['string_id'])]).transpose()
 
    hits = hits[hits[:,0].argsort()] 
    ind = []
    for i in range(hit_num):
        if hits[i,0]<0: ind.append(i)
        else:break
    hits = np.delete(hits,ind,axis=0)
    hit_num = hit_num-len(ind)
    #print(f'####:{hit_num}')
    if interfilter == True:
        eff = np.load(efffile)
        map = np.isin(hits[:,4],eff)
        hits = hits[map]
        hits = hits[:,0:4]
        hit_num= len(hits)
    else:
        hits = hits[:,0:4]
    print("There is",hit_num,"hit(s) in this event.")
    #print(hits)
    fenergy = simu.loc[id][1]['injection_energy']
    fzenith = simu.loc[id][1]['injection_zenith']
    fazimuth = simu.loc[id][1]['injection_azimuth']
    feventid = tf.cast(counter,dtype=tf.int64)
    fhitnum = tf.cast(hit_num,dtype = tf.int64) 
    hits_input = hits.flatten()
    if(hit_num>1300000):
        return 0
    stbinding = knndatap.knndatap(hits_input,hit_num)

    
    if bool(stbinding)==False:
        return 0
    
    tmin = hits[0,0]
    tmax = hits[-1,0]
    fmean = np.mean(hits,axis=0)
    fcc = np.concatenate(([tmin,tmax],fmean),axis = 0)

    #print(tmp)
    hit_sources = stbinding[1::3]
    hit_targets = stbinding[2::3]
    metrics = np.array(stbinding[3::3])
    metrics= metrics[:,np.newaxis]
    #for i in range(5):
        #print(hit_sources[i],hit_targets[i])

    #print("tmp",len(tmp))
    #print('**********')
    hit_adjacency = tfgnn.Adjacency.from_indices(source=("hit",tf.cast(hit_sources,dtype=tf.int32)),target=("hit",tf.cast(hit_targets,dtype=tf.int32)))
    #print("@@@@@@@@@@")
    ###generate GT###
    hit = tfgnn.NodeSet.from_fields(
        sizes = [hit_num],
        features={ 
            "4Dvec":tf.cast(hits,dtype=tf.float32),
        })
    coincidence = tfgnn.EdgeSet.from_fields(
        sizes = tf.shape(hit_sources),
        features={
            "metric": tf.cast(metrics,dtype=tf.float32)
        },
        adjacency=hit_adjacency)

    context = tfgnn.Context.from_fields(
        features={"injection_zenith":[fzenith],"event_energy":[fenergy],"injection_azimuth":[fazimuth],"event_id":[feventid],"hit_num":[fhitnum],"cc":tf.cast(fcc,dtype=tf.float32)})

    graphtensor = tfgnn.GraphTensor.from_pieces(node_sets={"hit":hit},edge_sets={"coincidence":coincidence},context=context)
    #print("%%%%%%%%%%%")
    return graphtensor



In [ ]:


##############################
counter =0
with tf.io.TFRecordWriter('/Users/mac/simu/debug/SAM') as writer:
        simulationpath = '/Users/mac/simu/debug/MuMinus_Hadrons_seed_1_meta_data.parquet'
        simu = rp(simulationpath)
        print(f'#######begin file#######')
        for i in range(5000):
            graph = makeGT(i,counter)
            if graph ==0:
                continue
            else:
                counter+=1
                example = tfgnn.write_example(graph)
                writer.write(example.SerializeToString())
        print(f'Now we have {counter} events in file')

#################################
print(counter)